#### Adapted from "Attention on MNIST (Saliency and grad-CAM)" from the keras-vis package

https://github.com/raghakot/keras-vis/blob/master/examples/mnist/attention.ipynb

In [1]:
import numpy as np
import keras
from matplotlib import pyplot as plt
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Activation, Input
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
from vis.visualization import visualize_saliency
from tqdm import tqdm
from keras.models import load_model
from keras.utils.data_utils import get_file
from vis.utils import utils
from keras import activations
from scipy.ndimage.filters import gaussian_filter

Using TensorFlow backend.
/Users/jegpeek/anaconda3/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


Load the trained model. Here were are looking at the model trained on data that have not had the fixed fourier power applied.

In [2]:
#model_path = get_file('DLfile.dms','https://www.dropbox.com/s/46z20173319zicv/model2_128_4.dms?dl=0')
#load_model(model_path)
model = load_model('model2_128_4.dms')

In [3]:
img_rows = 128
img_cols = 128

#loading test (non-FFP) data:
test_x_b1 = np.load('b1p.01XY_test_ss1_sh_64_ims_128.npy')
test_x_b01 = np.load('b.1p.01XY_test_ss1_sh_64_ims_128.npy')

teshape = test_x_b1.shape

x_test = np.zeros([teshape[2]*2, img_rows, img_rows])
x_test[::2, :, :] = np.transpose(test_x_b01)
x_test[1::2, :, :] = np.transpose(test_x_b1)

y_test = np.zeros([teshape[2]*2])
y_test[::2] = np.zeros([teshape[2]])
y_test[1::2] = np.ones([teshape[2]])


For saliency to work properly the last layer needs to be linear, not softmax

In [4]:
layer_idx = utils.find_layer_idx(model, "preds")

# Swap softmax with linear
model.layers[layer_idx].activation = activations.linear
model = utils.apply_modifications(model)

In [22]:
low_am = 1
label=''
if low_am == 1:
    label = '_low_AM'
plt.figure(figsize=[16, 16])
# just to get these sorted by columns, not rows
colsfirst = np.reshape(np.arange(36), [6, 6]).T.flatten()
for i in tqdm(np.arange(18)):
    idx = i*2+1
    class_idx =1
    plt.subplot(6,6,colsfirst[i]+1)
    grads = visualize_saliency(model, layer_idx, filter_indices=class_idx, seed_input=(x_test[idx])[:, :, None], backprop_modifier='guided')
    plt.imshow(x_test[idx][...], cmap='gray')
    # filtering here smooths out some of the substructure caused by CNN architecture
    gfilt = gaussian_filter(np.sum(grads, axis=2), 1.0)
    plt.contour(gfilt, cmap='bwr_r', levels=[200], linewidths=1)
    #plt.imshow(masked_gfilt, cmap='plasma')
    plt.xticks([])
    plt.yticks([])
for i in tqdm(np.arange(18,36)):
    idx = i*2
    class_idx =0
    plt.subplot(6,6,colsfirst[i]+1)
    grads = visualize_saliency(model, layer_idx, filter_indices=class_idx, seed_input=(x_test[idx])[:, :, None], backprop_modifier='guided')
    plt.imshow(x_test[idx][...], cmap='gray')
    # filtering here smooths out some of the substructure caused by CNN architecture
    gfilt = gaussian_filter(np.sum(grads, axis=2), 1.0)
    plt.contour(gfilt, cmap='bwr_r', levels=[200], linewidths=1)
    #plt.imshow(masked_gfilt, cmap='plasma')
    plt.xticks([])
    plt.yticks([])
plt.savefig('Saliency_both.pdf')

|----------| 0/18   0% [elapsed: 00:00 left: ?, ? iters/sec]

/Users/jegpeek/anaconda3/lib/python3.5/site-packages/matplotlib/contour.py:1180: UserWarning: No contour levels were found within the data range.
  warnings.warn("No contour levels were found"
                                                                     